In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from kaggle.competitions import nflrush

In [ ]:
train_df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', low_memory=False)

In [ ]:
MEDIAN_YARDS = train_df["Yards"].median()
print(f"The median value of train target is {MEDIAN_YARDS} yards.")

**We are going to make a constant median prediction with sigmoidal post-processing. Inspired by [this Notebook](https://www.kaggle.com/hukuda222/nfl-simple-evluation-trick) by @hukuda222.**

In [ ]:
def sigmoid(x):
    return 1. / (1 + np.exp(-x))

plt.plot(range(-99, 100), [1 if i >= 0 else 0 for i in range(-99, 100)], label="actual")
plt.plot(range(-99, 100), [sigmoid(i) for i in range(-99, 100)], label="predicted")
plt.legend();

In [ ]:
y_train = train_df.loc[range(0, 509762, 22), "Yards"]

In [ ]:
y_pred, y_true = np.zeros((len(y_train), 199)), np.zeros((len(y_train), 199))

for i in range(len(y_train)):
    # all predictions are the same :)
    y_pred[i, :] = sigmoid(0.42 * (np.arange(-99, 100) - MEDIAN_YARDS))

for i, p in enumerate(y_train):
    for j in range(-99, 100):
        if j >= p:
            y_true[i, j + 99] = 1

print("Validation score:", np.sum(np.power(y_pred - y_true, 2)) / (199 * len(y_train)))

In [ ]:
env = nflrush.make_env()

for (test_df, sample_prediction_df) in tqdm_notebook(env.iter_test()):
    
    # all predictions are the same :)
    y_test_pred = sigmoid(0.42 * (np.arange(-99, 100) - MEDIAN_YARDS))
    
    env.predict(pd.DataFrame(data=[y_test_pred], columns=sample_prediction_df.columns))
    
env.write_submission_file()